# BIKE - Bit Flipping Key Encapsulation

- Criação de um protótipo em Sagemath para o algoritmo **BIKE**.
- Pretende-se implementar um **KEM**, que seja **IND-CPA** seguro, e um **PKE** que seja **IND-CCA** seguro.

## Protótipo

In [240]:
from sage.all import *
# noinspection PyUnresolvedReferences
from sage.modules.vector_mod2_dense import Vector_mod2_dense

### Parâmetros

Parâmetros para o nível de segurança 1

In [263]:
r = 257  # 12323  # Comprimento do bloco (block length)
n = r * 2  # Comprimento do código (code length)
w = 142  # Peso da linha (row weight)
t = 134  # Peso do erro (error weight)
l = 256  # Comprimento do segredo partilhado (shared secret size) | NOTA: Este parametro é fixo para todos os níveis de segurança

# BGF decoder parameters - nível de segurança 1
NbIter = 5  # Número de iterações do decoder
tau = 3  # Threshold Gap | TODO: Confirmar se este comentário está correto
threshold = lambda S, _i: max(floor(0.0069722 * S + 13.530), 36)  # Threshold function

In [264]:
F = GF(2)

M = F ** l  # Message space

R = PolynomialRing(F, 'x')
x = R.gen()
Rr = QuotientRing(R, R.ideal(x ** r - 1))  # Polynomial ring R / (x^r - 1)

KK = F ** l  # Private key space

print("Message space M:   ", M)
print("Shared key space K:", KK)
print("Polynomial ring R: ", R)
print("Quotient ring Rr:  ", Rr)

MElement = type(M.random_element())  # Basicamente binário
RElement = type(Rr.random_element())  # Elemento de Rr
KElement = type(KK.random_element())  # Basicamente binário

print("MElement:", MElement)
print("RElement:", RElement)
print("KElement:", KElement)

Message space M:    Vector space of dimension 256 over Finite Field of size 2
Shared key space K: Vector space of dimension 256 over Finite Field of size 2
Polynomial ring R:  Univariate Polynomial Ring in x over Finite Field of size 2 (using GF2X)
Quotient ring Rr:   Univariate Quotient Polynomial Ring in xbar over Finite Field of size 2 with modulus x^257 + 1
MElement: <class 'sage.modules.vector_mod2_dense.Vector_mod2_dense'>
RElement: <class 'sage.rings.polynomial.polynomial_quotient_ring.PolynomialQuotientRing_generic_with_category.element_class'>
KElement: <class 'sage.modules.vector_mod2_dense.Vector_mod2_dense'>


### Funções auxiliares

In [265]:
def generate_sparse(weight: int, size: int) -> RElement:
    """
    Gera um sparse vector.
    Entrada: weight - número de elementos não nulos (Hamming weight)
             size - tamanho do vector
    Saída: elemento de Rr
    """
    while True:
        # Generate a random list of size 'size' with 'weight' non-zero elements
        sparse_rep = [0] * size
        for _ in range(weight):
            rand_index = randint(0, size - 1)
            while sparse_rep[rand_index] != 0:
                rand_index = randint(0, size - 1)

            sparse_rep[rand_index] = 1

        assert sum(sparse_rep) == weight
        return Rr(sparse_rep)

In [266]:
def bytes_to_bits(b: bytes) -> list:
    assert type(b) == bytes

    return [int(bit) for byte in b for bit in bin(byte)[2:].zfill(8)]

In [267]:
def expand(lis: list, size: int) -> list:
    assert type(lis) == list

    return lis + [0] * (l - len(lis))

In [268]:
# noinspection PyPep8Naming
def R_to_bytes(r: RElement) -> bytes:
    assert type(r) == RElement

    return bytes(r.list())


# noinspection PyPep8Naming
def bytes_to_R(b: bytes) -> RElement:
    assert type(b) == bytes

    return Rr(list(b))


assert bytes_to_R(R_to_bytes(Rr([1, 0, 1]))) == Rr([1, 0, 1])


# noinspection PyPep8Naming
def M_to_bytes(m: MElement) -> bytes:
    assert type(m) == MElement

    bits = m.list()
    bit_string = ''.join(str(bit) for bit in bits)  # convert the list of bits to a string

    return int(bit_string, 2).to_bytes(len(bits) // 8, byteorder='big')


# noinspection PyPep8Naming
def bytes_to_M(b: bytes) -> MElement:
    assert type(b) == bytes

    bytess = expand(bytes_to_bits(b), l)

    assert len(bytess) == l

    return M(bytess)


assert bytes_to_M(M_to_bytes(M([1, 0] * (l // 2)))) == M([1, 0] * (l // 2))

In [269]:
def getHammingWeight(m: MElement) -> int:
    acc = 0
    for i in m:
        if i == 1:
            acc += 1

    return acc


assert getHammingWeight(M([1, 0] * (l // 2))) == l // 2

In [270]:
def xor(a: MElement, b: MElement) -> MElement:
    assert len(a) == len(b)
    return M([a[i] ^ b[i] for i in range(len(a))])

### Funções de Hash necessárias

#### Função H

In [271]:
# noinspection PyPep8Naming
def H(m: MElement) -> (RElement, RElement):
    assert type(m) == MElement
    # TODO: Migrate this to use AES256-CTR PRNG

    e0 = generate_sparse(t, r)
    e1 = generate_sparse(t, r)

    return e0, e1


H(M([1, 0] * (l // 2)))

(xbar^254 + xbar^253 + xbar^251 + xbar^250 + xbar^249 + xbar^247 + xbar^246 + xbar^245 + xbar^243 + xbar^242 + xbar^241 + xbar^238 + xbar^237 + xbar^236 + xbar^235 + xbar^233 + xbar^232 + xbar^231 + xbar^229 + xbar^228 + xbar^227 + xbar^223 + xbar^222 + xbar^220 + xbar^217 + xbar^215 + xbar^213 + xbar^208 + xbar^207 + xbar^205 + xbar^201 + xbar^200 + xbar^197 + xbar^196 + xbar^195 + xbar^194 + xbar^192 + xbar^190 + xbar^189 + xbar^188 + xbar^187 + xbar^186 + xbar^184 + xbar^183 + xbar^182 + xbar^180 + xbar^178 + xbar^177 + xbar^175 + xbar^173 + xbar^170 + xbar^169 + xbar^167 + xbar^164 + xbar^163 + xbar^162 + xbar^161 + xbar^160 + xbar^156 + xbar^154 + xbar^153 + xbar^148 + xbar^147 + xbar^146 + xbar^145 + xbar^144 + xbar^143 + xbar^141 + xbar^140 + xbar^139 + xbar^138 + xbar^137 + xbar^135 + xbar^134 + xbar^130 + xbar^129 + xbar^128 + xbar^125 + xbar^124 + xbar^123 + xbar^120 + xbar^119 + xbar^117 + xbar^115 + xbar^114 + xbar^113 + xbar^112 + xbar^111 + xbar^108 + xbar^107 + xbar^105 

#### Função L

In [288]:
# noinspection PyPep8Naming
def L(e0: RElement, e1: RElement) -> MElement:
    assert type(e0) == RElement
    assert type(e1) == RElement

    # Apply the SHA384 hash function to the concatenation of e0 and e1
    from hashlib import sha384

    m = sha384()

    m.update(R_to_bytes(e0))

    m.update(R_to_bytes(e1))

    digest = m.digest()

    # Concat all the bits of the digest into a list of bits
    digest = bytes_to_bits(digest[-l // 8:])  # We only need l bits (l / 8 bytes)

    return M(digest)  # Returns the MElement corresponding to the digest


L(Rr([1, 0, 1]), Rr([1, 0, 1]))

(0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1)

#### Função K

In [289]:
# noinspection PyPep8Naming
def K(m: MElement, c0: RElement, c1: MElement) -> KElement:
    assert type(m) == MElement
    assert type(c0) == RElement
    assert type(c1) == MElement

    # Apply the SHA384 hash function to the concatenation of m, c0 and c1
    from hashlib import sha384

    digest = sha384(M_to_bytes(m) + R_to_bytes(c0) + M_to_bytes(c1)).digest()

    digest = bytes_to_bits(digest[:l // 8])  # We only need l bits (l / 8 bytes)

    return KK(digest)  # Returns the KElement corresponding to the digest


K(M([1, 0] * 128), Rr([1, 0, 1]), M([1, 0] * 128))

(1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0)

### Função de computação do sindrome (syndrome computation)

In [290]:
def compute_syndrome(c0: RElement, h0: RElement) -> RElement:
    assert type(c0) == RElement
    assert type(h0) == RElement

    return c0 * h0

### Geração de chaves

In [291]:

def keygen() -> ((RElement, RElement), MElement, RElement):
    """
    Geração de chaves
    Entrada: Nenhum
    Saída: (pk, sk)
    """
    h0 = generate_sparse(w // 2, l)
    h1 = generate_sparse(w // 2, l)

    sigma = M.random_element()

    h0_inv = 1 / h0
    h = h1 * h0_inv

    return (h0, h1), sigma, h

In [292]:
# Teste TODO: Mover isto para a secção de testes

(priv_key, sigma, public_key) = keygen()

# Print the hex representation of the public key and secret key
print("public_key: ", public_key.lift())

public_key:  x^253 + x^249 + x^248 + x^246 + x^242 + x^241 + x^240 + x^238 + x^237 + x^233 + x^232 + x^231 + x^230 + x^227 + x^222 + x^219 + x^218 + x^215 + x^212 + x^209 + x^207 + x^205 + x^203 + x^201 + x^198 + x^197 + x^194 + x^191 + x^189 + x^188 + x^187 + x^186 + x^184 + x^181 + x^180 + x^179 + x^177 + x^176 + x^175 + x^173 + x^169 + x^167 + x^166 + x^163 + x^161 + x^160 + x^159 + x^157 + x^156 + x^153 + x^149 + x^143 + x^138 + x^137 + x^136 + x^135 + x^126 + x^121 + x^120 + x^119 + x^117 + x^116 + x^114 + x^111 + x^110 + x^107 + x^106 + x^103 + x^102 + x^100 + x^97 + x^96 + x^95 + x^93 + x^90 + x^85 + x^83 + x^82 + x^81 + x^80 + x^76 + x^75 + x^72 + x^69 + x^68 + x^67 + x^65 + x^63 + x^62 + x^57 + x^56 + x^54 + x^51 + x^50 + x^46 + x^45 + x^40 + x^34 + x^30 + x^29 + x^25 + x^24 + x^21 + x^20 + x^17 + x^16 + x^15 + x^11 + x^9 + x^8 + x^7 + x^4 + x^3


### Encapsulamento

In [293]:
def calculate_c(e0: RElement, e1: RElement, h: RElement, seed: MElement) -> (RElement, MElement):
    assert type(e0) == RElement
    assert type(e1) == RElement
    assert type(h) == RElement
    assert type(seed) == MElement

    return e0 + e1 * h, seed + L(e0, e1)

In [294]:
def encapsulate(h: RElement) -> (KElement, (RElement, MElement)):
    assert type(h) == RElement

    seed: MElement = M.random_element()
    (e0, e1) = H(seed)

    c = calculate_c(e0, e1, h, seed)
    c0, c1 = c

    k = K(seed, c0, c1)

    return k, c

In [295]:
# Teste TODO: Mover isto para a secção de testes

(priv_key, sigma, public_key) = keygen()

(k, c) = encapsulate(public_key)

print("k: ", k.lift())
print("c: ", c)

k:  (0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1)
c:  (xbar^255 + xbar^251 + xbar^250 + xbar^249 + xbar^246 + xbar^245 + xbar^243 + xbar^239 + xbar^238 + xbar^236 + xbar^235 + xbar^234 + xbar^232 + xbar^229 + xbar^227 + xbar^226 + xbar^223 + xbar^216 + xbar^215 + xbar^214 + xb

### Desencapsulamento

In [296]:
# noinspection PyUnresolvedReferences
from sage.matrix.matrix_mod2_dense import Matrix_mod2_dense
# noinspection PyUnresolvedReferences
from sage.matrix.matrix_integer_dense import Matrix_integer_dense


def decoder(x: RElement, h0: RElement, h1: RElement) -> (RElement, RElement):
    assert type(x) == RElement
    assert type(h0) == RElement
    assert type(h1) == RElement

    print("decoder function")

    # Convert x to a vectorSpace element
    x = RElement_to_VectorSpace(x)

    H_mat = get_H_matrix(h0, h1)

    return BGF(x, H_mat)


def BGF(s: Vector_mod2_dense, H: Matrix_mod2_dense) -> (RElement, RElement):
    assert type(s) == Vector_mod2_dense
    assert type(H) == Matrix_mod2_dense

    print("BGF function")
    e: Vector_mod2_dense = copy(VectorSpace(GF(2), n).zero())
    d = w // 2

    HTranspose = H.transpose()

    for i in range(1, NbIter + 1):
        T = threshold(getHammingWeight(s + e * HTranspose), i)
        e, black, gray = BFIter(s + e * HTranspose, e, T, H)
        if i == 1:
            e = BFMaskedIter(s + e * HTranspose, e, black, ((d + 1) // 2) + 1, H)
            e = BFMaskedIter(s + e * HTranspose, e, gray, ((d + 1) // 2) + 1, H)

    if s == e * HTranspose:
        (e0, e1) = e[:r], e[r:]
        return e0, e1
    else:
        return Rr(0), Rr(0)


def BFIter(s: Vector_mod2_dense, e: Vector_mod2_dense, T: int, H: Matrix_mod2_dense) -> (RElement, RElement, RElement):
    """
    Black-Gray-Flip (BGF) BFIter function.
    :param s: the syndrome vector
    :param e: the error vector
    :param T: the threshold
    :param H: the parity-check matrix
    :return: a tuple containing the updated error vector, the set of black bits, and the set of gray bits
    """
    assert type(s) == Vector_mod2_dense
    assert type(e) == Vector_mod2_dense
    assert type(T) == int
    assert type(H) == Matrix_mod2_dense

    n = H.ncols()
    black = copy(VectorSpace(GF(2), n).zero())
    gray = copy(VectorSpace(GF(2), n).zero())

    for j in range(n):
        if ctr(H, s, j) >= T:
            e[j] += 1
            black[j] = 1
        elif ctr(H, s, j) >= T - tau:
            gray[j] = 1

    return e, black, gray


def ctr(H: Matrix_mod2_dense, s: Vector_mod2_dense, j: int) -> int:
    """
    ctr(H; s; j). This function computes a quantity referred to as the counter (aka the number of unsatisfied parity-checks) of j.
    It is the number of ’1’ (set bits) that appear in the same position in the syndrome s and in the j-th column of the matrix H.
    """
    assert type(H) == Matrix_mod2_dense
    assert type(s) == Vector_mod2_dense
    assert type(j) == int

    return getHammingWeight(s.pairwise_product(H.column(j)))


def BFMaskedIter(s: Vector_mod2_dense, e: Vector_mod2_dense, mask: Vector_mod2_dense, T: int,
                 H: Matrix_mod2_dense) -> RElement:
    """
    Black-Gray-Flip (BGF) BFMaskedIter function.
    :param s: the syndrome vector
    :param e: the error vector
    :param mask: the mask vector
    :param T: the threshold
    :param H: the parity-check matrix
    :return: the updated error vector
    """
    assert type(s) == Vector_mod2_dense
    assert type(e) == Vector_mod2_dense
    assert type(mask) == Vector_mod2_dense
    assert type(T) == int
    assert type(H) == Matrix_mod2_dense

    n = H.ncols()

    for j in range(n):
        if ctr(H, s, j) >= T:
            e[j] = e[j] + mask[j]

    return e

In [297]:
def RElement_to_VectorSpace(element: RElement) -> Vector_mod2_dense:
    assert type(element) == RElement

    elem_coefs = element.lift().list()

    v = vector(GF(2), elem_coefs + [0] * (r - len(elem_coefs)))

    return v

In [298]:
def get_H_matrix(h0: RElement, h1: RElement) -> Matrix_integer_dense:
    assert type(h0) == RElement

    print("get_H_matrix function")

    H = block_matrix(1, 2, [get_circulant_matrix(h0), get_circulant_matrix(h1)])

    assert H.dimensions() == (r, n)

    return H


def get_circulant_matrix(element: RElement) -> Matrix_mod2_dense:
    assert type(element) == RElement

    print("get_circulant_matrix function")
    vec = element.lift().list()
    # Fill the rest of the vector with zeros
    vec = vec + [0] * (r - len(vec))

    circ = matrix.circulant(vec)

    return circ

In [299]:
def decapsulate(h0: RElement, h1: RElement, sigma: MElement, c0: RElement, c1: MElement) -> KElement:
    assert type(h0) == RElement
    assert type(h1) == RElement
    assert type(sigma) == MElement
    assert type(c0) == RElement
    assert type(c1) == MElement

    e_ = decoder(c0 * h0, h0, h1)

    m_ = c1 + L(e_[0], e_[1])

    if e_ == H(m_):
        return K(m_, c0, c1)
    else:
        return K(sigma, c0, c1)

In [303]:
# Teste TODO: Mover isto para a secção de testes

(priv_key, sigma, public_key) = keygen()

(k, (c0, c1)) = encapsulate(public_key)

k_ = decapsulate(*priv_key, sigma, c0, c1)

print("k: ", k)
print("k_: ", k_)

print("Total bits: ", l)
print("Different bits: ", getHammingWeight(k - k_))

decoder function
get_H_matrix function
get_circulant_matrix function
get_circulant_matrix function
BGF function
k:  (1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0)
k_:  (0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1

## KEM (Key Encapsulation Mechanism) - IND-CPA (INDistinguishable under Chosen Plaintext Attack)

## PKE (Public Key Encryption) - IND-CCA (INDistinguishable under Chosen Ciphertext Attack)

In [285]:
def encrypt(h: RElement, m: MElement) -> (RElement, MElement):
    e0 = generate_sparse(t, r)
    e1 = generate_sparse(t, r)

    c0 = e0 + e1 * h
    c1 = m + L(e0, e1)

    return c0, c1

In [180]:
def decrypt(h0: RElement, h1: RElement, s: RElement) -> (RElement, RElement):
    return decoder(s * h0, h0, h1)

In [301]:
(priv_key, sigma, public_key) = keygen()

message = "Hello World!"
messageM = bytes_to_M(bytes(message, 'utf-8'))

(c0, c1) = encrypt(public_key, messageM)

m_ = decrypt(priv_key, c0, c1)

TypeError: 'sage.modules.vector_mod2_dense.Vector_mod2_dense' object cannot be interpreted as an integer